# 학습목표
  1. graphlab create를 사용한 추천

* Movie lens 평점 데이터 셋
 - http://grouplens.org/datasets/movielens/100k/

* load user, rating, items

In [1]:
import pandas as pd

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('ml-100k/u.user', sep='|', names=u_cols,
 encoding='latin-1')

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=r_cols,
 encoding='latin-1')

i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/u.item', sep='|', names=i_cols,
 encoding='latin-1')

In [2]:
users

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
5,6,42,M,executive,98101
6,7,57,M,administrator,91344
7,8,36,M,administrator,05201
8,9,29,M,student,01002
9,10,53,M,lawyer,90703


In [3]:
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,6,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,01-Jan-1995,NaN,http://us.imdb.com/Title?Yao+a+yao+yao+dao+wai...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,7,Twelve Monkeys (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Twelve%20Monk...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
7,8,Babe (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Babe%20(1995),0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
8,9,Dead Man Walking (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Dead%20Man%20...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,10,Richard III (1995),22-Jan-1996,NaN,http://us.imdb.com/M/title-exact?Richard%20III...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [4]:
ratings

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [ ]:
users.shape

In [ ]:
ratings.shape

In [ ]:
items.shape

* load train / test data

In [5]:
cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings_train = pd.read_csv('ml-100k/ua.base', sep='\t', names = cols, encoding='latin-1')
ratings_test = pd.read_csv('ml-100k/ua.test', sep='\t', names = cols, encoding='latin-1')


In [ ]:
print ratings_train.shape
print ratings_test.shape

* graphlab register & download
 - https://turi.com/download/academic.html
 - email, key 발급
 - e.g) https://turi.com/download/install-graphlab-create-command-line.html?email=macmath22%40gmail.com&key=3F03-279E-306F-D384-CBF7-1252-XXXX-XXXX

* train / test sframe 생성

In [6]:
import graphlab

In [7]:
train_data = graphlab.SFrame(ratings_train)
test_data = graphlab.SFrame(ratings_test)

This non-commercial license of GraphLab Create for academic use is assigned to aaron@homemaster.co.kr and will expire on March 31, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1490949166.log


In [8]:
train_data

user_id,movie_id,rating,timestamp
1,1,5,874965758
1,2,3,876893171
1,3,4,878542960
1,4,3,876893119
1,5,3,889751712
1,6,5,887431973
1,7,4,875071561
1,8,1,875072484
1,9,5,878543541
1,10,3,875693118


* popularity based recommendation

In [9]:
popularity_model = graphlab.popularity_recommender.create(train_data, 
                                                        user_id='user_id', 
                                                        item_id='movie_id', 
                                                          target='rating')

Recsys training: model = popularity

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

Data prepared in: 0.099056s

90570 observations to process; with 1680 unique items.

In [10]:
popularity_recomm = popularity_model.recommend(users = range(1, 6), k = 10)
popularity_recomm.print_rows(num_rows = 50)

+---------+----------+-------+------+
| user_id | movie_id | score | rank |
+---------+----------+-------+------+
|    1    |   1656   |  5.0  |  1   |
|    1    |   1536   |  5.0  |  2   |
|    1    |   1500   |  5.0  |  3   |
|    1    |   1293   |  5.0  |  4   |
|    1    |   1599   |  5.0  |  5   |
|    1    |   1467   |  5.0  |  6   |
|    1    |   1201   |  5.0  |  7   |
|    1    |   1189   |  5.0  |  8   |
|    1    |   1122   |  5.0  |  9   |
|    1    |   814    |  5.0  |  10  |
|    2    |   1656   |  5.0  |  1   |
|    2    |   1536   |  5.0  |  2   |
|    2    |   1500   |  5.0  |  3   |
|    2    |   1293   |  5.0  |  4   |
|    2    |   1599   |  5.0  |  5   |
|    2    |   1467   |  5.0  |  6   |
|    2    |   1201   |  5.0  |  7   |
|    2    |   1189   |  5.0  |  8   |
|    2    |   1122   |  5.0  |  9   |
|    2    |   814    |  5.0  |  10  |
|    3    |   1656   |  5.0  |  1   |
|    3    |   1536   |  5.0  |  2   |
|    3    |   1500   |  5.0  |  3   |
|    3    | 

연습문제)
 1. ratings_train에서 평균 평점이 가장 높은 영화 10개를 출력하세요

In [11]:
ratings_train.groupby('movie_id')['rating'].mean().sort_values(ascending = False).head(10)

movie_id
1500    5.0
1293    5.0
1122    5.0
1189    5.0
1656    5.0
1201    5.0
1599    5.0
814     5.0
1467    5.0
1536    5.0
Name: rating, dtype: float64

* Collaborative Filtering 사용
 - User-based, Item-based
 - 유사도를 이용하여 weighted-sum을 계산

* Graphlab에서 지원하는 유사도 
 1. Jaccard
 2. Cosine
 3. Pearson
 https://turi.com/products/create/docs/generated/graphlab.recommender.item_similarity_recommender.ItemSimilarityRecommender.html

In [12]:
item_sim_model = graphlab.item_similarity_recommender.create(train_data, 
                                                        user_id='user_id', 
                                                        item_id='movie_id', 
                                                        target='rating', 
                                                similarity_type='cosine')

item_sim_recomm = item_sim_model.recommend(users=range(1,6), k = 10)
popularity_recomm.print_rows(num_rows = 50)

Recsys training: model = item_similarity

Warning: Ignoring columns timestamp;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 90570 observations with 943 users and 1680 items.

...

| 158.139ms                           | 100              | 1680            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.18487s

In [13]:
item_sim_recomm.print_rows(50)

+---------+----------+----------------+------+
| user_id | movie_id |     score      | rank |
+---------+----------+----------------+------+
|    1    |   423    | 0.983400806671 |  1   |
|    1    |   202    | 0.949590799235 |  2   |
|    1    |   655    | 0.788672750005 |  3   |
|    1    |   403    | 0.765623665038 |  4   |
|    1    |   568    | 0.751179529283 |  5   |
|    1    |   385    | 0.722872027914 |  6   |
|    1    |   265    | 0.648438795605 |  7   |
|    1    |   393    | 0.624321931646 |  8   |
|    1    |   357    | 0.579889190333 |  9   |
|    1    |   483    | 0.500958802818 |  10  |
|    2    |    50    | 1.12562584877  |  1   |
|    2    |   181    | 1.06517731685  |  2   |
|    2    |    7     | 0.999819083856 |  3   |
|    2    |   121    | 0.941627963231 |  4   |
|    2    |    9     | 0.831989913033 |  5   |
|    2    |    98    | 0.822288922392 |  6   |
|    2    |   117    | 0.792605129572 |  7   |
|    2    |    56    | 0.707731382205 |  8   |
|    2    |  

* evaluation
 - recall & precision
 - recall : 사용자가 좋아한다고 한 아이템을 몇개나 추천했는가?
 - precision : 추천된 아이템 중에서 몇개나 좋아하는가?
 
 - Recall만 고려할 경우, recall을 올리기 위해서는 모든 아이템을 추천하면 됨
 - 이 경우, precision이 현저하게 낮아짐
 - 높은 precision과 recall이 동시에 요구됨
 - 가장 이상적인 경우 precision = recal = 1

In [14]:
model_performance = graphlab.compare(test_data, [popularity_model, item_sim_model])
graphlab.show_comparison(model_performance,[popularity_model, item_sim_model])

PROGRESS: Evaluate model M0

Precision and recall summary statistics by cutoff
+--------+-------------------+-------------------+
| cutoff |   mean_precision  |    mean_recall    |
+--------+-------------------+-------------------+
|   1    |        0.0        |        0.0        |
|   2    |        0.0        |        0.0        |
|   3    | 0.000353481795688 | 0.000106044538706 |
|   4    | 0.000265111346766 | 0.000106044538706 |
|   5    | 0.000212089077413 | 0.000106044538706 |
|   6    | 0.000176740897844 | 0.000106044538706 |
|   7    | 0.000151492198152 | 0.000106044538706 |
|   8    | 0.000265111346766 | 0.000212089077413 |
|   9    | 0.000235654530458 | 0.000212089077413 |
|   10   | 0.000212089077413 | 0.000212089077413 |
+--------+-------------------+-------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1

Precision and recall summary statistics by cutoff
+--------+----------------+-----------------+
| cutoff | mean_precision |   mean_recall   |
+--------+---

* graphlab classification & regression example
 - yelp review data

In [15]:
import graphlab as gl
data =  gl.SFrame('https://static.turi.com/datasets/regression/yelp-data.csv')

Downloading https://static.turi.com/datasets/regression/yelp-data.csv to /var/tmp/graphlab-aaronbyun/3726/b013c8cb-d440-4d61-88b4-8dda1be2a364.csv

Finished parsing file https://static.turi.com/datasets/regression/yelp-data.csv

Parsing completed. Parsed 100 lines in 0.748841 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,str,int,str,str,str,dict,int,int,int,list,str,str,float,float,str,int,int,float,str,str,float,str,int,str,int,int,int,dict]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 47948 lines. Lines per second: 32663.3

Finished parsing file https://static.turi.com/datasets/regression/yelp-data.csv

Parsing completed. Parsed 215879 lines in 4.16025 secs.

In [16]:
data

business_id,date,review_id,stars,text,type
9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here onmy birthday for break ...,review
ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why somepeople give bad reviews ...,review
6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Riceis so good and I also ...,review
_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVEChaparral Dog Park!!! ...",review
6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager ScottPetello is a good egg!!! ...,review
-yxfBYGB6SEqszmxJxd97A,2007-12-13,m2CKSsepBCoRYWxiRUsxAg,4,"Quiessence is, simplyput, beautiful. Full ...",review
zp713qNhx8d9KCJJnrw1xA,2010-02-12,riFQ3vxNpP4rWLk_CSri2A,5,Drop what you're doingand drive here. After I ...,review
hW0Ne_HTHEAgGF1rAdmR-g,2012-07-12,JL7GXJ9u4YMx7Rzs05NfiQ,4,"Luckily, I didn't have totravel far to make my ...",review
wNUea3IXZWD63bbOQaOH-g,2012-08-17,XtnfnYmnJYi71yIuGsXIUA,4,Definitely come for Happyhour! Prices are amaz ...,review
nMHhuYan8e3cONo3PornJA,2010-08-11,jJAIXA46pU1swYyRCdfXtQ,5,Nobuo shows his uniquetalents with everything ...,review


In [17]:
data['is_good'] = data['stars'] >= 3

train_data, test_data = data.random_split(0.8)

model = gl.classifier.create(train_data, target='is_good',
                             features = ['user_avg_stars',
                                         'business_avg_stars',
                                         'user_review_count',
                                         'business_review_count'])

predictions = model.classify(test_data)

results = model.evaluate(test_data)


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.

PROGRESS: The following methods are available for this type of problem.
PROGRESS: BoostedTreesClassifier, RandomForestClassifier, DecisionTreeClassifier, SVMClassifier, LogisticClassifier
PROGRESS: The returned model will be chosen according to validation accuracy.


Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 164015

Number of classes           : 2

...

| 5         | 6        | 0.756119     | 0.867238          | 0.870254            |

| 6         | 7        | 0.902925     | 0.867238          | 0.870254            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

PROGRESS: Model selection based on validation accuracy:
PROGRESS: ---------------------------------------------
PROGRESS: BoostedTreesClassifier          : 0.870253920555
PROGRESS: RandomForestClassifier          : 0.869671583176
PROGRESS: DecisionTreeClassifier          : 0.869089245796
PROGRESS: SVMClassifier                   : 0.167831
PROGRESS: LogisticClassifier              : 0.870254
PROGRESS: ---------------------------------------------
PROGRESS: Selecting LogisticClassifier based on validation set performance.


In [18]:
graphlab.linear_regression

model = gl.regression.create(train_data, target='stars',
                                    features = ['user_avg_stars',
                                                'business_avg_stars',
                                                'user_review_count',
                                                'business_review_count'])


predictions = model.predict(test_data)
print predictions
results = model.evaluate(test_data)
print results

PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



Boosted trees regression:

--------------------------------------------------------

Number of examples          : 163905

Number of features          : 4

...

| 4         | 0.248424     | 3.857579           | 3.457291             | 1.252184      | 1.259326        |

| 5         | 0.328476     | 3.777940           | 3.377651             | 1.111055      | 1.117141        |

| 6         | 0.408932     | 3.737615           | 3.337326             | 1.034569      | 1.040013        |

+-----------+--------------+--------------------+----------------------+---------------+-----------------+

[4.8690409660339355, 4.38094425201416, 4.542483806610107, 3.809565544128418, 4.0548014640808105, 4.500245094299316, 2.998141288757324, 3.666382074356079, 0.9990736842155457, 3.7152841091156006, 4.084072113037109, 3.7705495357513428, 4.563923358917236, 3.608267307281494, 2.9683799743652344, 2.7248265743255615, 3.3660736083984375, 1.0367276668548584, 4.691277027130127, 3.7934963703155518, 4.256025314331055, 3.5475590229034424, 3.412891149520874, 2.2469964027404785, 3.8056418895721436, 3.185636043548584, 3.282209873199463, 3.410905122756958, 2.3719706535339355, 1.9907877445220947, 2.4845848083496094, 2.4845848083496094, 4.854714870452881, 2.987093925476074, 4.851530075073242, 3.410905122756958, 3.9621834754943848, 4.0921831130981445, 4.164196014404297, 3.7184648513793945, 4.861121654510498, 4.773089408874512, 3.7343225479125977, 3.935849666595459, 3.87731671333313, 0.9990736842155457, 4.4183549880981445, 2.9367434978485107, 2.830758571624756, 3.9190316200256348, 3.8945200443267822, 4.8690